## 윗 파트

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 47.3 MB 1.5 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 8.6 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595726 sha256=fa554e7f66616ee8947ed82b33f86947d444412280d2232b3c485ca9283138a6
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 9.2 MB/s 
     |████████████████████████████████| 769 kB 8.2 MB/s 
     |████████████████████████████████| 3.0 MB 57.5 MB/s 
     |████████████████████████████████| 895 kB 71.3 MB/s 


In [ ]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-rs99xhvv
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-rs99xhvv
     |████████████████████████████████| 131 kB 7.9 MB/s 
     |████████████████████████████████| 4.5 MB 64.9 MB/s 
     |████████████████████████████████| 3.4 MB 47.8 MB/s 
     |████████████████████████████████| 67 kB 6.8 MB/s 
     |████████████████████████████████| 3.3 MB 59.5 MB/s 
     |████████████████████████████████| 596 kB 71.2 MB/s 
     |████████████████████████████████| 8.5 MB 73.1 MB/s 
     |████████████████████████████████| 79 kB 11.2 MB/s 
     |████████████████████████████████| 127 kB 77.2 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15449 sha256=3263f514c52ca20be9cc7b2da8f0695a6cb9e8f999871d3e446085e8aef3b89a
  Stored in directory: /tmp/pip-ephem-wheel-cache-7y27y64o/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd0

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

## 내 데이터

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/AIB_PJ2/dataset/kobert_train_data_final.csv')

In [ ]:
train_data.head()

,Unnamed: 0,comments,label
0,1916,남한테 관심을 갖기 않았으니까 남한테서도 관심을 못 받는 모양이지. 딱히 누가 손해...,1.0
1,1318,아이구.. 어제 늦게까지 무리하신 게 화근이었을까요 편히 푹 쉬셔요 너무 늦지 않게...,2.0
2,38479,잘려고 누웠어.,1.0
3,26023,괜찮아?,1.0
4,26714,그리고 이건 선금. 현금으로 백 만원 넣었어요.,1.0


In [ ]:
train_data = train_data.drop('Unnamed: 0', axis=1)
train_data['label'] = train_data['label'].astype(int)

In [ ]:
train_data.head()

,comments,label
0,남한테 관심을 갖기 않았으니까 남한테서도 관심을 못 받는 모양이지. 딱히 누가 손해...,1
1,아이구.. 어제 늦게까지 무리하신 게 화근이었을까요 편히 푹 쉬셔요 너무 늦지 않게...,2
2,잘려고 누웠어.,1
3,괜찮아?,1
4,그리고 이건 선금. 현금으로 백 만원 넣었어요.,1


In [ ]:
train_data.dtypes

comments    object
label        int64
dtype: object

## 동기님 데이터

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/AIB_PJ2/dataset/dg_ko_data.csv')

In [ ]:
train_data.dtypes

label        int64
comments    object
dtype: object

In [ ]:
train_data = train_data[['comments', 'label']]

In [ ]:
train_data['comments'] = train_data['comments'].astype(str)

In [ ]:
train_data.head()

,comments,label
0,약간흔들림이 있지만 아담한사이즈에 디자인 이쁘고 가격대비 가성비 짱입니다,0
1,옛추억이 잠겨 정말 재미있게 플레이 하였습니다,1
2,얘,0
3,1점도 아깝다 내 느낌은 정품이 아닌것 같은 느낌 무척 아쉽네요,0
4,퐁신퐁신하고 사진그대로에요 커버패드느낌,1


## 모델 훈련

In [ ]:
def make_data_list(data):
  data_list = []
  for q, label in zip(data['comments'], data['label'])  :
      data = []
      data.append(q)
      data.append(str(label))

      data_list.append(data)
  return data_list

In [ ]:
data_list = make_data_list(train_data)

In [ ]:
data_list

[['남한테 관심을 갖기 않았으니까 남한테서도 관심을 못 받는 모양이지. 딱히 누가 손해를 보는 것도 아니라서 나는 그걸로 좋았어.',
  '1'],
 ['아이구.. 어제 늦게까지 무리하신 게 화근이었을까요 편히 푹 쉬셔요 너무 늦지 않게 주무시길 바랄게요 감사하고 사랑해요 오늘도 우주님꺼예요 평생💋💜',
  '2'],
 ['잘려고 누웠어.', '1'],
 ['괜찮아?', '1'],
 ['그리고 이건 선금. 현금으로 백 만원 넣었어요.', '1'],
 ['긴 하루가 이제 끝났어요 보고싶어요 사랑해요 편한 밤 되시길..💜', '2'],
 ['먹을거로 때리기가능한가요', '0'],
 ['핸드폰., 집에 놓고 왔다며?', '1'],
 ['빨리 끝내고 찜질방으로 오세요! 안 그러면 언니랑 나랑 다 먹어 치울 거예요.', '1'],
 ['이거 봤냐?', '1'],
 ['....카메라 따위 필요 없어.', '1'],
 ['그럼 외람되지만, 부인의 유품이라도 잠시 보여주시면.. ', '1'],
 ['니를 포기 안할라고 이 난리하는 거 아냐!', '0'],
 ['그래요. 그럼 내일 봐요.', '1'],
 ['어허~ 과장, 그렇게 안봤는데 몸 사리는건가?', '1'],
 ['거기까지 굴러갔어요? 몰랐어요 죄송..', '1'],
 ['급하면 지놈들이 다시 연락할 거 아니야. 공중전화 있는데 가서 공중전화로 통화를 해야한다고.', '1'],
 ['엄마의 마음은 얼어죽을....', '1'],
 ['그러죠.', '1'],
 ['개그맨이겠지.', '1'],
 ['저도 많이 보고 싶어요 쓰담쓰담..', '2'],
 ['니가 차려 먹어. 난 짐 정리할거야.', '1'],
 ['아직도 화 났네.', '1'],
 ['ㅋㅋㅋㅋ좋은 점심', '2'],
 ['좀 지난 거긴 한데.. 기사거리론 충분 할 거야.', '1'],
 ['적당했습니다.', '1'],
 ['어! 500만원 부주!! 니 결혼자금 2천!! 또!! 생각난다!', '1'],
 ['그렇다니까요. 아니!! 솔직히 말해서. 이혼 사유 놓고

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 32 # 32 로 바꿔보기
warmup_ratio = 0.1
num_epochs = 3 # 3으로 해보기
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5 # 0.00005 5e-5, 4e-5, 3e-5, 2e-5중 하나 가장 좋은 것

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
data_train[0] # 마지막 array 는 attention mask sequence이다. 

(array([   2, 3093,  517,   54,  517,   54, 1100, 7086, 5850,  517,   54,
         517,   54,    3,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(14, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1534 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.9484850764274597 train acc 0.59375
epoch 1 batch id 201 loss 0.6378909349441528 train acc 0.8247823383084577
epoch 1 batch id 401 loss 0.2511270344257355 train acc 0.8452306733167082
epoch 1 batch id 601 loss 0.3715919256210327 train acc 0.8544613144758736
epoch 1 batch id 801 loss 0.2692527174949646 train acc 0.8617353308364545
epoch 1 batch id 1001 loss 0.45434364676475525 train acc 0.8654470529470529
epoch 1 batch id 1201 loss 0.3241853415966034 train acc 0.8684689841798501
epoch 1 batch id 1401 loss 0.25617483258247375 train acc 0.869825124910778
epoch 1 train acc 0.8708849413298566


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 2 test acc 0.8817138671875


  0%|          | 0/1534 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.3648640811443329 train acc 0.84375
epoch 3 batch id 201 loss 0.24540601670742035 train acc 0.9001865671641791
epoch 3 batch id 401 loss 0.06998461484909058 train acc 0.9008728179551122
epoch 3 batch id 601 loss 0.13561978936195374 train acc 0.9048460898502496
epoch 3 batch id 801 loss 0.18625642359256744 train acc 0.9094101123595506
epoch 3 batch id 1001 loss 0.33674880862236023 train acc 0.9119630369630369
epoch 3 batch id 1201 loss 0.07249274849891663 train acc 0.9137177352206495
epoch 3 batch id 1401 loss 0.08730093389749527 train acc 0.9150383654532477
epoch 3 train acc 0.9149486636245111


  0%|          | 0/512 [00:00<?, ?it/s]

epoch 3 test acc 0.88052978515625


In [ ]:
# 입력전 토큰화 해주기
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

    return np.argmax(logits)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
predict("안녕")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


1

In [ ]:
test_data = pd.read_csv("/content/drive/MyDrive/AIB_PJ2/dataset/[jh]cleaned_data_new.csv", keep_default_na=False)
test_data = test_data[['comments', 'file']] #만약 comments가 공백일경우, 이미지가 있다는 얘기니깐 레이블에는 1을 추가해준다.

In [ ]:
len(test_data)

10297

In [ ]:
label_li = []
for i in range(len(test_data)):
  if test_data['comments'][i] == '':
    label_li.append(1)
  elif "참여했습니다." in test_data['comments'][i]:
    label_li.append(1)
  elif "나갔습니다." in test_data['comments'][i]:
    label_li.append(0)
  else :
    label_li.append(predict(test_data['comments'][i]))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
len(label_li), len(test_data)

(10297, 10297)

In [ ]:
test_data['label'] = label_li

In [ ]:
test_data.to_csv('[jh]labeled_data_0113.csv')

In [ ]:
test_data2 = pd.read_csv("/content/drive/MyDrive/AIB_PJ2/dataset/[jh]cleaned_data_new.csv", keep_default_na=False)

In [ ]:
test_data2['label'] = label_li

In [ ]:
test_data2.to_csv('[jh]labeled_data_0113_new.csv')

참고 : 
https://velog.io/@seolini43/KOBERT%EB%A1%9C-%EB%8B%A4%EC%A4%91-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0-%ED%8C%8C%EC%9D%B4%EC%8D%ACColab   
데이터 : https://aihub.or.kr/aidata/7978

In [ ]:
test_data

,comments,file,label
0,님이 참여했습니다.,,1
1,사랑해요🖤,,2
2,님이 참여했습니다.,,1
3,님이 참여했습니다.,,1
4,단톡 전용 인스타 대체일지... 자려고했는데 파들..,,1
...,...,...,...
10292,ㅎㅎ 오늘도 비가 주륵주륵~ 벼보리님 우산 잘 쓰고 다니셔요 !,,1
10293,Operator오늘은 ~ 수요일 ~ 스팀겜 하는 날~ 일주일의 절반이 벌써 다가왔네...,,1
10294,재밌는시간 보내셨겠네요 ㅎㅎ\n저는 학교갔는데 생각했던것보단 정겹고 좋네요 은근 ㅎ...,,1
10295,보리쟝이무화과를 먹었으니 저는 복숭아를 먹겠읍니다 후후\n푹 쉬고 7시에 방송에서...,,1


In [ ]:
# torch.save(model, 'my_model.pt')